In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

/home/arman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
N = 100000
X = np.random.normal(size=(N, 4)).astype(np.float32)
y = (X[:, 0] < X[:, 1]) & (X[:, 1] < X[:, 2]) & (X[:, 2] < X[:, 3])
y = np.array([int(x) for x in y]).astype(np.float32)

In [12]:
TRAIN_SIZE = int(N*0.8)
train_x, test_x = X[:TRAIN_SIZE], X[TRAIN_SIZE:]
train_y, test_y = y[:TRAIN_SIZE], y[TRAIN_SIZE:]

In [13]:
def train_input_fn(features, labels, batch_size=128):
    # Can do all features as one:
    # dataset = tf.data.Dataset.from_tensor_slices(({'feats': features}, labels))
    # Or separate "columns":
    dataset = tf.data.Dataset.from_tensor_slices(({'feat0': features[:,0],
                                                   'feat1': features[:, 1],
                                                   'feat2': features[:, 2],
                                                   'feat3': features[:, 3]},
                                                   labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset

In [14]:
def eval_input_fn(features, labels, batch_size=32):
    feats = {'feat0': features[:,0], 'feat1': features[:, 1],
             'feat2': features[:, 2],'feat3': features[:, 3]}
    
    if labels is None:
        inputs = feats
    else:
        inputs = (feats, labels)

    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    dataset = dataset.batch(batch_size)

    return dataset

In [15]:
my_feature_columns = []
for key in ['feat0', 'feat1', 'feat2', 'feat3']:
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [16]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[128, 128, 64, 64],
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpki6d3mze', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2662fcd400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
classifier.train(input_fn=lambda: train_input_fn(train_x, train_y, batch_size=100), steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpki6d3mze/model.ckpt.
INFO:tensorflow:loss = 68.22771, step = 0
INFO:tensorflow:global_step/sec: 498.814
INFO:tensorflow:loss = 3.2198677, step = 100 (0.201 sec)
INFO:tensorflow:global_step/sec: 760.287
INFO:tensorflow:loss = 1.2451464, step = 200 (0.133 sec)
INFO:tensorflow:global_step/sec: 717.866
INFO:tensorflow:loss = 3.7431018, step = 300 (0.138 sec)
INFO:tensorflow:global_step/sec: 698.384
INFO:tensorflow:loss = 2.0196223, step = 400 (0.143 sec)
INFO:tensorflow:global_step/sec: 685.542
INFO:tensorflow:loss = 0.6388005, step = 500 (0.147 sec)
INFO:tensorflow:global_step/sec: 677.698
INFO:tensorflow:loss = 0.07393646, step = 600 (0.146 sec)
INFO:tensorflow:global_step/sec: 682.307
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 733.665
INFO:tensorflow:loss = 0.067370586, step = 8000 (0.137 sec)
INFO:tensorflow:global_step/sec: 674.471
INFO:tensorflow:loss = 0.0082531655, step = 8100 (0.147 sec)
INFO:tensorflow:global_step/sec: 783.79
INFO:tensorflow:loss = 0.075205855, step = 8200 (0.128 sec)
INFO:tensorflow:global_step/sec: 470.089
INFO:tensorflow:loss = 0.03061301, step = 8300 (0.214 sec)
INFO:tensorflow:global_step/sec: 462.679
INFO:tensorflow:loss = 0.013933196, step = 8400 (0.216 sec)
INFO:tensorflow:global_step/sec: 637.846
INFO:tensorflow:loss = 0.31938633, step = 8500 (0.156 sec)
INFO:tensorflow:global_step/sec: 661.712
INFO:tensorflow:loss = 0.09543728, step = 8600 (0.151 sec)
INFO:tensorflow:global_step/sec: 760.84
INFO:tensorflow:loss = 0.47998136, step = 8700 (0.131 sec)
INFO:tensorflow:global_step/sec: 638.129
INFO:tensorflow:loss = 0.016848862, step = 8800 (0.158 sec)
INFO:tensorflow:global_step/sec: 681.014
INFO:tensorflow:loss = 0.04701033, step = 8900 (0.146 s

In [18]:
eval_result = classifier.evaluate(input_fn=lambda: eval_input_fn(test_x, test_y, batch_size=100))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-04:58:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpki6d3mze/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-04:58:06
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.99935, accuracy_baseline = 0.95705, auc = 0.99999076, auc_precision_recall = 0.99979293, average_loss = 0.0018778242, global_step = 10000, label/mean = 0.04295, loss = 0.18778242, precision = 0.9895833, prediction/mean = 0.04329745, recall = 0.99534345

Test set accuracy: 0.999



In [19]:
print("full eval results")
print(eval_result)

full eval results
{'accuracy': 0.99935, 'accuracy_baseline': 0.95705, 'auc': 0.99999076, 'auc_precision_recall': 0.99979293, 'average_loss': 0.0018778242, 'label/mean': 0.04295, 'loss': 0.18778242, 'precision': 0.9895833, 'prediction/mean': 0.04329745, 'recall': 0.99534345, 'global_step': 10000}
